# Censo de poblacion y vivienda 2020

Esta libreta contiene un análisis inicial de las columnas del formato del Censo de población y vivienda del año 2020 realizado por el Instituto de Geografia y Estadistica (INEGI) para toda la Republica Méxicana. Corrigue los errores de formato detectados y exporta los diferentes niveles de información que contienen la tabla (Nacional, Estatal, Municipal y Localidad) en un formato adecuado para su importación a bases de datos SQL y su utilización en Sistemas de Información Geográfica.

## Librerias y configuraciones

In [2]:
import os
import pandas as pd
import random as rnd
import numpy as np
from math import ceil


In [3]:
# Variable definida para imprimir información adicional en la libreta.
verbose = True

## Importar archivos

In [4]:
# DATA_DIR es una variable de entorno en mi computadora a la cual se le 
# asignó el valor de el dicrectorio donde se guardan los datos a procesar.
data_dir = os.getenv('DATA_DIR')

# La información censal la guardo en una carpeta llamada EventosCensales 
# dentro del directorio de datos.
censales_dir = 'EventosCensales'

# Se respetó la estructura de carpetas original que se obtiene al descomprimir 
# el archivo zip que se obtiene de INEGI. Algunas veces se modifican estos 
# archivos asi que pueden cambiar. En este caso el zip se descomprimió en una 
# carpeta llamada iter_00_cpv2020.
censo_2020_dir = 'iter_00_cpv2020'

# Los datos se encuentran dentro de la carpeta conjunto_de_datos y el descriptor 
# de cada uno de los campos se encuentra en diccionario_datos.
subdir_datos = 'conjunto_de_datos'
subdir_descriptor = 'diccionario_datos'

# El nombre del archivo donde se encentran los datos es 
# 'conjunto_de_datos_iter_00CSV20.csv' y descriptor de la base tiene el nombre 
# 'diccionario_datos_iter_00CSV20.csv'. Los dos vienen en formato csv.
archivo_datos = 'conjunto_de_datos_iter_00CSV20.csv'
archivo_descriptor = 'diccionario_datos_iter_00CSV20.csv'

# Concatenar las rutas de archivos
ruta_de_archivo = os.path.join(data_dir, 
                               censales_dir, 
                               censo_2020_dir, 
                               subdir_datos, 
                               archivo_datos)

ruta_de_descriptor = os.path.join(data_dir, 
                                  censales_dir, 
                                  censo_2020_dir, 
                                  subdir_descriptor, 
                                  archivo_descriptor)

if verbose:

    print(f'El archivo de datos existe: {os.path.isfile(ruta_de_archivo)}')
    print(f'El descriptor de base de datos existe: {os.path.isfile(ruta_de_descriptor)}')


El archivo de datos existe: True
El descriptor de base de datos existe: True


In [5]:
# Importar archivo de datos
raw_censo_2020 = pd.read_csv(ruta_de_archivo)

C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\2739558870.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_censo_2020 = pd.read_csv(ruta_de_archivo)


In [6]:
# Importar descriptor de datos
nuevos_nombres = {
    'Núm.':'NUM', 
    'Indicador':'INDICADOR',
    'Descripción':'DESCRIPCION',
    'Mnemónico':'NOMCOL',
    'Rangos':'RANGOS',
    'Longitud':'LONGITUD'
}

descriptor = pd.read_csv(ruta_de_descriptor, skiprows=4, usecols=list(nuevos_nombres.keys()))
descriptor.rename(columns=nuevos_nombres, inplace=True)

## Análisis inicial

La mayor parte de las columnas se importa como objeto, lo que significa que existen diferentes tipos de datos dentro de cada una de las columnas. Para homogeneizar los datos se tiene que hacer un analisis mas detallado para definir que tipo de dato debe de ser cada una de las columnas y cuales son los valores que no son de este tipo en cada una de ellas.

In [7]:
# Primera descripcion
raw_censo_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 286 entries, ENTIDAD to TAMLOC
dtypes: int64(6), object(280)
memory usage: 426.9+ MB


El primer registro es un buen acercamiento al tipo de dato que contiene cada una de las columnas. Representa el total nacional, y como tal hay columnas que no aplican. Tal es el caso de las columnas que definen el nombre y clave del Estado, Municipio, y Localidad. No contienen tampoco longitud y latiud, estos datos solo estan presentes en los registros que representan localidades especificas.

In [8]:
# Diccionario cuyas llaves son el nombre de columna y valores del primer registro 
# del dataframe.
ejemplo_datos = {key : value for key, value in 
                zip(raw_censo_2020.columns, raw_censo_2020.iloc[0])}

if verbose:
    print(f'Numero de columnas {len(ejemplo_datos.items())}')

Numero de columnas 286


In [9]:
# En general las libretas jupyter no imprimen un texto de mas de ciertas lineas.
# para ver todos los registros hay que cambiar los valores dentro de los
# corchetes ([0:20]) de la linea comentada para recorrerlos todos. Hay 286 pares 
# llave: valor en el diccionario.

#for item in list(ejemplo_datos.items())[0:20]: print(item)  

Existen tres tipos de columnas que deben estar en formato string. El primer tipo es el que contiene los nombres de los Estados, Municipios y Localidades. El segundo tipo contiene las claves de los diferentes niveles de entidad. Y el tercero contiene las coodenadas geograficas en formato de grados minutos y seguntos. Las columnas que deberian ser texto son las siguientes:

['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD', 'LATITUD']

El resto de las columnas contiene valores numericos. Casi todas tienen el formato de numeros enteros pero 6 que contienen promedios y una que contiene la relacion que existe entre hombres y mujeres de la delimitación geoestadistica. Estos campos contienen decimales por lo que deben de tener el formato Float. Las columnas con valores decimales son:

['REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 'PROM_OCUP', 'PRO_OCUP_C']

El resto de las columnas contienen numeros enteros.

In [10]:
# Todas las columnas
columnas_censo_2020 = list(raw_censo_2020.columns)

# Columnas de texto
columnas_string = ['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 
                   'LONGITUD', 'LATITUD']
# Columnas con valores decimales
columnas_decimal = ['REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 
                    'PROM_OCUP', 'PRO_OCUP_C']

# El resto de las columnas son numeros enteros
columnas_entero = [i for i in columnas_censo_2020 
                   if i not in columnas_string + columnas_decimal]

Existen dos valores no numericos en las columnas con formato de numeros enteros y tambien de decimales. Estos valores hacen que el programa represente como objeto todos los registros. Los valores de texto que hay que sustituir son los siguientes:

['*', 'N/D']

La columna ALTITUD tiene 32 valores no numericos. En teoria deberia contener valores enteros y va a ser tratado como un caso aparte.

In [11]:
# Tarda un poco en calcularse esta celda
valores_string_por_columna = {}
for column in columnas_censo_2020:
        # Se quitaron las columnas con nombres pero se dejaron las columnas con claves
        # alfanumericas
        if column not in ['NOM_ENT', 'NOM_MUN', 'NOM_LOC', 'LONGITUD', 'LATITUD']:
                valores_no_numericos = tuple(raw_censo_2020[~raw_censo_2020[column]
                                                            .astype('string')
                                                            .str.strip()
                                                            .str.replace('.', '', regex = False)
                                                            .str.isnumeric()]
                                                            [column].unique())
                if valores_no_numericos in valores_string_por_columna.keys():
                        valores_string_por_columna[valores_no_numericos] += [column]
                
                else:
                        valores_string_por_columna[valores_no_numericos] = [column]


In [12]:
# Registros con valores de texto
#raw_censo_2020[raw_censo_2020.eq('*').any(1)]
#raw_censo_2020[raw_censo_2020.eq('N/D').any(1)]


In [13]:
# Resumen valores de texto

if verbose:
    ND_verdaderos = raw_censo_2020.eq('N/D').any(axis = 1).value_counts()[1]
    asterisco_verdaderos = raw_censo_2020.eq('*').any(axis = 1).value_counts()[1]
    
    elementos_por_linea = 7
    for llave, valor in valores_string_por_columna.items():
        iteraciones_llaves = len(llave) / elementos_por_linea
        iteraciones_valores = len(valor) / elementos_por_linea

        if iteraciones_llaves < 1: iteraciones_llaves  = 1
        else: iteraciones_llaves = ceil(iteraciones_llaves)

        if iteraciones_valores < 1: iteraciones_valores = 1
        else: iteraciones_valores = ceil(iteraciones_valores)

        
        inicial, final = 0, elementos_por_linea
        
        print('Columnas:')
        for columnas_similares in range(0, iteraciones_valores):

            print(valor[inicial: final])
            inicial += elementos_por_linea
            final += elementos_por_linea

        print('Valores de texto:')   
        inicial, final = 0, elementos_por_linea
        for valores_llave in range(0, iteraciones_llaves):

            print(llave[inicial:final])
            inicial += elementos_por_linea
            final += elementos_por_linea
        print()
    
    print(f'Filas en los que aparece el valor N/D: {ND_verdaderos}')
    print(f'Filas en los que aparece el valor *: {asterisco_verdaderos}')

Columnas:
['ENTIDAD', 'MUN', 'LOC', 'POBTOT', 'VIVTOT', 'TVIVHAB']
Valores de texto:
()

Columnas:
['ALTITUD']
Valores de texto:
('-006', '-001', '-002', '-005', '00-2', '-012', '-007')
('-008', '-009', '-003', '-010', '00-1', '00-4', -8.0)
(-2.0, -6.0, -5.0, -1.0, -7.0, '-004', '00-3')
('-013', -3.0, -18.0, -4.0, -9.0, '00-8', '00-5')
(-10.0, -11.0, -12.0, -15.0)

Columnas:
['POBFEM', 'POBMAS', 'TAMLOC']
Valores de texto:
('*',)

Columnas:
['P_0A2', 'P_0A2_F', 'P_0A2_M', 'P_3YMAS', 'P_3YMAS_F', 'P_3YMAS_M', 'P_5YMAS']
['P_5YMAS_F', 'P_5YMAS_M', 'P_12YMAS', 'P_12YMAS_F', 'P_12YMAS_M', 'P_15YMAS', 'P_15YMAS_F']
['P_15YMAS_M', 'P_18YMAS', 'P_18YMAS_F', 'P_18YMAS_M', 'P_3A5', 'P_3A5_F', 'P_3A5_M']
['P_6A11', 'P_6A11_F', 'P_6A11_M', 'P_8A14', 'P_8A14_F', 'P_8A14_M', 'P_12A14']
['P_12A14_F', 'P_12A14_M', 'P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F']
['P_18A24_M', 'P_15A49_F', 'P_60YMAS', 'P_60YMAS_F', 'P_60YMAS_M', 'REL_H_M', 'POB0_14']
['POB15_64', 'POB65_MAS', 'P_0A4', 'P_0

Las columnas LONGITUD y LATITUD tiene la informacion de las coordenadas en formato de grados minutos y segundos. Existen registros sin un valor en estas columnas pero son registros que representan la informacion total nacional, de los estados, municipios y la informacion de localidades de una y dos viviendas.

Los registros que si contienen información en estas columnas contienen el mismo formato de texto. Una vez que se les quitan los espacios en blanco antes y despues de los textos (.strip), contienen dos o tres digitos seguidos del simbolo de grado (°), seguido de otros dos digitos y el simbolo de minutos ('), seguido de tres digitos mas, un punto (.), tres digitos mas y el simbolo de segundos ("), seguido de un espacio en blanco y la letra W en la longitud y N para la latitud.

In [14]:
#raw_censo_2020[raw_censo_2020.LATITUD.isna()].NOM_LOC.unique()
#raw_censo_2020[raw_censo_2020.LONGITUD.isna()].NOM_LOC.unique()

In [15]:
# Comprobar formato de longitud
raw_censo_2020.LATITUD\
    .str.strip()\
    .str.contains(r'^\d{2}°\d{2}\'\d{2}\.\d{3}\"\sN$')\
    .value_counts(dropna = False)

True    189432
NaN       6230
Name: LATITUD, dtype: int64

In [16]:
#Comprobar formato de latitud
raw_censo_2020.LONGITUD\
    .str.strip()\
    .str.contains(r'^\d{2,3}°\d{2}\'\d{2}\.\d{3}\"\sW$')\
    .value_counts(dropna = False)

True    189432
NaN       6230
Name: LONGITUD, dtype: int64

In [17]:
#descriptor.columns                   # Columnas del diccionario
#list(descriptor.NOMCOL.unique())[:]  # Columnas del iter

In [18]:
# Información de columnas del Censo 2020
columna_de_interes = 'ENTIDAD'  # Poner columna de interes aqui.
info_columna = descriptor[descriptor.NOMCOL.isin([columna_de_interes])]

for columna in info_columna.columns:
    print(f'{columna}:')
    print(info_columna[columna].values)


NUM:
[1]
INDICADOR:
['Clave de entidad federativa']
DESCRIPCION:
['Código que identifica a la entidad federativa. El código 00 identifica a los registros con los totales a nivel nacional.']
NOMCOL:
['ENTIDAD']
RANGOS:
['00…32']
LONGITUD:
[2]


## Cambiar el formato a columnas

Despues de limpiar los datos en todas las columnas y haber separado la información de coordenadas, quedarán 11 columnas en formato decimal (Float64), 275 en formato de numer entero (Int64), y 8 en formato de texto (string).

### Columnas de texto

Las columnas con las claves alfanumericas de los estados, municipios y localidades fueron importadas en su mayoria como numeros. Se van a cambiar a formato de numero y se les va a reformatear. para que tengan el numero correcto de caracterers necesario para poder concatenarlas.

Entidad es una clave que debe de tener 2 posiciones aunque su valor numerico sea 1. En este caso la clave correcta que debe de tener es '01'. La clave de los municipios esta conformada por 3 posiciones, y la de localidad por 4. Al igual que la clave del estado, si el municipio o la localidad tuvieran asignado el numero 1 para representarlos, las claves correctas deben de ser '001' y '0001' segun sea el caso.

In [19]:
# Reformatear columnas con claves alfa numericas.
for string_column, keylength in zip(['ENTIDAD', 'MUN', 'LOC'], [2, 3, 4]):
    raw_censo_2020[string_column] = raw_censo_2020[string_column].astype('string').str.zfill(keylength)



In [20]:
for columna in columnas_string:
    # Estas ya se cambiaron
    if columna not in ['ENTIDAD','MUN', 'LOC']:
        raw_censo_2020[columna] = raw_censo_2020[columna].astype('string').str.strip()

In [21]:
#raw_censo_2020.iloc[:,0:8].info()

In [22]:
# Resultado
if verbose:
    print('Claves de entidad, municipio y localidad corregidas:', end='\n\n')
    print(raw_censo_2020.iloc[rnd.randint(0,raw_censo_2020.shape[0]),0:8])

Claves de entidad, municipio y localidad corregidas:

ENTIDAD                      10
NOM_ENT                 Durango
MUN                         038
NOM_MUN        Vicente Guerrero
LOC                        0000
NOM_LOC     Total del Municipio
LONGITUD                   <NA>
LATITUD                    <NA>
Name: 58813, dtype: object


### Columnas numericas

Se sustituyeron los valores de texto en las columnas numericas con exepción de ALTITUD con el valor nulo (np.nan). Dependiento del tipo de dato que se definió ademas como numero entero ('Int64'), o como numero decimal ('Float64'). El resultado es 7 columnas decimales y 270 con numeros enteros despues de la columna de altitud.

In [23]:
# Sustituir valores de texto por valores nulos (np.nan) y definir el formato de
# entero o decimal segun corresponda.

for columna in raw_censo_2020.columns:
    # Qitar columnas de texto y Altitud
    if columna not in columnas_string + ['ALTITUD']:
        # Sustituir los valores de texto en columnas numericas por np.nan
        raw_censo_2020[columna].replace(to_replace = ['*', 'N/D'], value = np.nan, inplace = True)
        # Definir el tipo de columna a Float o a Integer según sea el caso.
        if columna in columnas_decimal:
            raw_censo_2020[columna] = raw_censo_2020[columna].astype('Float64')
        
        else:
            raw_censo_2020[columna] = raw_censo_2020[columna].astype('Int64')


In [24]:
# Resultado

# Todas las columnas despues del 9 son numericas. El nueve tambien es numerico
# pero representa la columna ALTITUD que sera analizada de manera individual
# mas adelante. 
raw_censo_2020.iloc[:, 9:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 277 entries, POBTOT to TAMLOC
dtypes: Float64(7), Int64(270)
memory usage: 465.2 MB


### Columnas de coordenadas

Las columnas LONGITUD Y LATITUD se conservarán tal y como se encuentran pero se procesaran de dos maneras para que existan columnas en el formato que los Sistemas de Información Geografica piden para geolocalizar puntos en el mapa. 

Los formatos que se aceptan son columnas numericas donde se definen de manera separada los grados, los minutos y los segundos para la longitud y la latiud. Y una sola columna en grados decimales, donde el numero entero representa los grados, y los digitos decimales representan la porcion de minutos y segundos restantes.

Antes, los programas GIS necestaban las coordenadas en grados decimales para poder renderizar el mapa. Hoy en dia tambien aceptan la información de grados, minutos y segundos en columnas aparte y lo calculan ellos de manera automatica. Se generarán las columnas para representar las localidades en los dos formatos.




In [25]:
# Caso grados, minutos, y segundos en columnas separadas.

# Separar columnas de longitud en grados, minutos, segundos y cardinalidad
longitud_separado = raw_censo_2020.LONGITUD.str.split(pat = '[°\'\"]', expand = True, regex = True)
latitud_separado = raw_censo_2020.LATITUD.str.split(pat = '[°\'\"]', expand = True, regex = True)

# Diccionarios con los nombres de las nuevas columnas
columnas_split_lon = {0:'LON_GRAD',1:'LON_MIN',2:'LON_SEG', 3:'LON_CARD'}
columnas_split_lat = {0:'LAT_GRAD',1:'LAT_MIN',2:'LAT_SEG',3:'LAT_CARD'}

# Renombrar las columnas
longitud_separado.rename(columns=columnas_split_lon, inplace=True)
latitud_separado.rename(columns=columnas_split_lat, inplace=True)


In [26]:
#longitud_separado.info()     # Tipo de columnas inicial
#latitud_separado.info()      # Tipo de columnas fincial

In [27]:
# Cambiar el formato de las columnas de longitud_separado
for columna in longitud_separado.columns:
    if columna in ['LON_GRAD', 'LON_MIN']:
        longitud_separado[columna] = longitud_separado[columna].astype('Int64')
    elif columna in ['LON_SEG']:
        longitud_separado[columna] = longitud_separado[columna].astype('Float64')
    elif columna in ['LON_CARD']:
        longitud_separado[columna] = longitud_separado[columna].astype('string')\
            .str.strip()
    else:
        print('Te equivocaste en algun nombre')


# Cambiar el formato de las columnas de latitud_separado
for columna in latitud_separado.columns:
    if columna in ['LAT_GRAD', 'LAT_MIN' ]:
        latitud_separado[columna] = latitud_separado[columna].astype('Int64')
    elif columna in ['LAT_SEG']:
        latitud_separado[columna] = latitud_separado[columna].astype('Float64')
    elif columna in ['LAT_CARD']:
        latitud_separado[columna] = latitud_separado[columna].astype('string')\
            .str.strip()
    else:
        print('Te equivocaste en algun nombre')


Grados decimales se obtienen sumando los grados a los minutos divididos entre 60 y los segundos divididos entre 3600.

In [28]:
# Caso Grados decimales

coordenadas_decimales = pd.DataFrame()

coordenadas_decimales['LON_DEC'] = longitud_separado.LON_GRAD + \
    (longitud_separado.LON_MIN / 60) + \
        (longitud_separado.LON_SEG / 3600)

coordenadas_decimales['LAT_DEC'] = latitud_separado.LAT_GRAD + \
    (latitud_separado.LAT_MIN / 60) + \
        (latitud_separado.LAT_SEG / 3600)

coordenadas_decimales['LON_CARD']= longitud_separado.LON_CARD

La cardinalidad influye en el signo de las coordenadas geograficas. Si la coordenada longitudinal esta en el lado Oeste (West - W)del meridiano de Greenwich tienen signo negativo. Si las coordenadas geograficas estan al sur del ecuador en la coordenada latitudinal tambien son negativas. Mexico esta arriba del ecuador pero al Oeste del meridiano de Greenwich. Por esta razon se multiplicaran las coordenadas longitudinales por -1.

In [29]:
# Convertir a negativo los grados, minutos, y segundos longitudinales al 
# Oeste del meridiano de Greenwich

for columna in ['LON_GRAD', 'LON_MIN', 'LON_SEG']:
    longitud_separado.loc[longitud_separado['LON_CARD'] == 'W', columna] = \
        longitud_separado.loc[longitud_separado['LON_CARD'] == 'W', columna] * -1


In [30]:
# Se podia de esta misma manera en los grados y minutos separados porque no hay
# localidades Mexicanas al Este del meridiano de Greenwich...
coordenadas_decimales['LON_DEC'] = coordenadas_decimales['LON_DEC'] * -1

In [31]:
longitud_separado.drop('LON_CARD', axis = 1, inplace = True)
latitud_separado.drop('LAT_CARD', axis = 1, inplace = True)

coordenadas_decimales.drop('LON_CARD', axis = 1, inplace = True)

In [32]:
#  Comprobaciones
#(longitud_separado > 0).value_counts() # 
#(coordenadas_decimales['LON_DEC'] > 0).value_counts()
#longitud_separado.columns
#latitud_separado.columns

In [33]:
#longitud_separado.info()     # Tipo de columnas final
#latitud_separado.info()      # Tipo de columnas final

#raw_censo_2020.columns.get_loc('LONGITUD')   # en indice 6
#raw_censo_2020.columns.get_loc('LATITUD')    # en indice 7


In [34]:
# Agregar columnas corregidas a raw_censo_2020
insert_position = raw_censo_2020.columns.get_loc('LATITUD') + 1
for columna, valores in pd.concat([longitud_separado, latitud_separado, coordenadas_decimales]
                                  , axis = 1).items():

    raw_censo_2020.insert(insert_position, columna, valores)
    insert_position += 1


In [35]:
# Solo queda la columna ALTITUD como objeto
raw_censo_2020.select_dtypes(include=object).columns


Index(['ALTITUD'], dtype='object')

### Columna de altitud

Ningun registro tiene valores despues del punto decimal y hay 23 registros que no pudieron ser convertidos a valor numerico. Los valores de texto antes detectados son 32. Esto significa que 10 valores de texto tenian el formato necesario para que pandas pudiera interpretarlos como numero.

Los datos de la columna seran forzados a enteros, y los valores coercionados a nulos por este proceso seran revisados posteriormente. No son muchos, suponiendo que fueran los inicialmente detectados serian 32 de 189_409. Existen zonas en la superficie terrestre de México que estan abajo del nivel del mar, y es muy posible que sea un problema de formato pero que los valores numericos esten correctos. 

In [36]:
# Resumen ALTITUD
if verbose:
    valores_texto_altitud = list(valores_string_por_columna.keys())[1]
    total_valores = len(valores_texto_altitud)
    valores_a_imprimir = 6
    inicial, final = 0, valores_a_imprimir

    print(f'Total de valores de texto en columna ALTITUD: {total_valores}', end='\n\n')

    print('Valores de texto en la columna ALTITUD:', end = '\n\n')
    for lineas_imprimir in range(0, ceil(total_valores/valores_a_imprimir)):
        print(valores_texto_altitud[inicial:final])
        inicial += valores_a_imprimir
        final += valores_a_imprimir
    print()
    print('Remanentes de dividir entre 1 los valores de altitud de las localidades',
        'en los registros no nulos convertidos a numero:',
        end= '\n\n')
    print( 
        (pd.to_numeric(raw_censo_2020[raw_censo_2020.LONGITUD.notnull()].ALTITUD, errors='coerce') % 1)\
            .value_counts(dropna=False)
    )


Total de valores de texto en columna ALTITUD: 32

Valores de texto en la columna ALTITUD:

('-006', '-001', '-002', '-005', '00-2', '-012')
('-007', '-008', '-009', '-003', '-010', '00-1')
('00-4', -8.0, -2.0, -6.0, -5.0, -1.0)
(-7.0, '-004', '00-3', '-013', -3.0, -18.0)
(-4.0, -9.0, '00-8', '00-5', -10.0, -11.0)
(-12.0, -15.0)

Remanentes de dividir entre 1 los valores de altitud de las localidades en los registros no nulos convertidos a numero:

0.0    189409
NaN        23
Name: ALTITUD, dtype: int64


In [37]:
#(pd.to_numeric(raw_censo_2020['ALTITUD'], errors= 'coerce') % 1).value_counts()

raw_censo_2020['ALTITUD'] = pd.to_numeric(raw_censo_2020['ALTITUD'], errors= 'coerce').astype('Int64')

In [38]:
# Resultado
raw_censo_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 294 entries, ENTIDAD to TAMLOC
dtypes: Float64(11), Int64(275), string(8)
memory usage: 492.2 MB


## Corte de los datos

La información del censo esta ya en buenas condiciones. Se harán a continuación diferentes cortes de los datos contenidos en el dataframe limpio. La información va ser dividida segun nivel de informacion y software en el que va a ser utilizidad. 

### Corte según niveles de información

La información va a ser separada por nivel en nacional, estatal, municipal y localidad. Una vez realizada esta división habra que hacer algunas modificaciones a las tablas borrando algunas columnas y cambiando el nombre de otras para ajustar la información a cada nivel.

La informacion con la sumatoria de todas las localidades a nivel nacional se identifica con la clave '00' en la columna ENTIDAD porque no representa a ningun estado.

Los datos a nivel estatal se encuentran eliminando los datos con la clave '00' en la columna ENTIDAD y en las filas con la clave de municipio (MUN )'000'.

Las filas que corresponden a los datos con las sumas municipales se encuentran eliminando los registros con '00' en la columna entidad, y '000' en la columna MUN. Los registros que deben ser incluidos son los que contienen las claves '0000', '9999', y '9998' en la columna LOC.

Finalmente, los datos a nivel localidad son los que restan al eliminar la clave '00' en ENTIDAD, '000' en la columna MUN, y '0000', '9999', y '9998' de la columna LOC.



In [39]:
# Registros a nivel nacional
iter_nacional = raw_censo_2020[raw_censo_2020['ENTIDAD'] == '00']

# Registros a nivel estatal
iter_estatal = raw_censo_2020[(raw_censo_2020['ENTIDAD'] != '00') & \
                              (raw_censo_2020['MUN'] == '000')]

# Registros a nivel municipal
iter_municipal = raw_censo_2020[(raw_censo_2020['ENTIDAD'] != '00') & \
                                (raw_censo_2020['MUN'] != '000') & \
                                (raw_censo_2020['LOC'].isin(['0000', '9999', '9998']))]

# Registros a nivel localidad
iter_localidad = raw_censo_2020[(raw_censo_2020['ENTIDAD'] != '00') & \
                                (raw_censo_2020['MUN'] != '000') & \
                                (~raw_censo_2020['LOC'].isin(['0000', '9999', '9998']))]



### Modificaciones según nivel de información

Existen columnas sin información alguna despues de haber separado los registros. Solo los datos a nivel localidad tienen información en todas las columnas una vez que fueron separados.

A nivel nacional, estatal y municipal la información de longitud y latitud es inexistente por lo que las columnas que fueron creadas anteriormente tambien estaran vacias. A estas columnas hay que agregar tamaño de localidad a los tres niveles (TAMLOC), ENTIDAD, NOM_ENT, MUN y NOM_MUN a nivel nacional, y MUN y NOM_MUN a nivel estatal.

El nombre de las columnas de LOC y NOM_LOC será modificado en los niveles nacional, estatal y muncipal porque las claves que contienen representan las sumatorias de la información en localidades de una sola vivienda (9998), de dos viviendas (9999), y de todas las localidades juntas (0000).

A la columna LOC se le asignara 'CV_SUMLOC', que representa 'Clave de Suma de localidades segun numero de vivienda y total'. A la NOM_LOC _se le asignará el nombre DS_SUMLOC y contiene el nombre de las diferentes sumatorias que se realzan en esas filas.

A nivel localidad y municipal se concatenara una clave que represente a cada entidad de manera única. Para los municipios esto se logra concatenando la columnas columnas con la clave del estado (ENTIDAD) y la del municipio (MUN). Para las localidades se tiene que agregar la columna con la clave de la localidad (LOC). El nombre de las columnas será CVE_GEO_M y CVE_GEO_L para la tabla de municipios y localidades respectivamente.






In [40]:
# Análisis iter_nacional
remover_columnas_nal = ['LONGITUD', 'LATITUD', 'LON_GRAD', 'LON_MIN', 'LON_SEG', 
                        'LAT_GRAD', 'LAT_MIN', 'LAT_SEG', 'LON_DEC', 'LAT_DEC', 
                        'ALTITUD', 'TAMLOC', 'ENTIDAD', 'MUN', 'NOM_MUN', 'NOM_ENT']

#iter_nacional.isnull().all().value_counts()                # 12 columnas no tienen ningun valor
#iter_nacional.isnull().all()[iter_nacional.isnull().all() == True].index  
                                                            # Columnas que no tienen registros

#iter_nacional[['ENTIDAD', 'MUN', 'LOC']].value_counts()    # Todos los valores son iguales en las
                                                            # columnas ENTIDAD y MUN, la columna
                                                            # LOC contiene tres valores

#iter_nacional.NOM_LOC.value_counts()                       # Las clave de LOC representan la suma
                                                            # de los datos segun el numero de
                                                            # viviendas que hay en la localidad.                                                          


In [41]:
# Transformaciones iter_nacional

# Remover columnas
iter_nacional.drop(columns= remover_columnas_nal, axis=1, inplace=True)

# Renombrar columnas
iter_nacional.rename(columns={'LOC':'CV_SUMLOC', 'NOM_LOC':'DS_SUMLOC'}, inplace = True)

C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\2825823151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iter_nacional.drop(columns= remover_columnas_nal, axis=1, inplace=True)
C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\2825823151.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iter_nacional.rename(columns={'LOC':'CV_SUMLOC', 'NOM_LOC':'DS_SUMLOC'}, inplace = True)


In [42]:
# Resultados y comprobaciones iter_nacional

# iter_nacional.columns   # Columnas fueron removidas

In [43]:
# Análisis iter_estatal

remover_columnas_ent = ['LONGITUD', 'LATITUD', 'LON_GRAD', 'LON_MIN', 'LON_SEG', 
                        'LAT_GRAD', 'LAT_MIN', 'LAT_SEG', 'LON_DEC', 'LAT_DEC', 
                        'ALTITUD', 'TAMLOC', 'MUN', 'NOM_MUN']

#iter_estatal.isnull().all().value_counts()             # 12 columnas no tienen ningun valor
#iter_estatal.isnull().all()[iter_nacional.isnull().all() == True].index  
                                                        # Columnas que no tienen registros

#iter_estatal['ENTIDAD'].value_counts().index           # ENTIDAD contiene valores del 01 - 32 que
                                                        # representa a cada uno de los estados

#iter_estatal[['MUN', 'LOC']].value_counts()            # Todos los valores en MUN son iguales
                                                        # y no representan nada. 

In [44]:
# Transformaciones iter_estatal

# Remover columnas
iter_estatal.drop(columns=remover_columnas_ent, axis=1, inplace=True)

# Renombrar columnas
iter_estatal.rename(columns={'LOC':'CV_SUMLOC', 'NOM_LOC':'DS_SUMLOC'}, inplace = True)


C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\3468801214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iter_estatal.drop(columns=remover_columnas_ent, axis=1, inplace=True)
C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\3468801214.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iter_estatal.rename(columns={'LOC':'CV_SUMLOC', 'NOM_LOC':'DS_SUMLOC'}, inplace = True)


In [45]:
# Resultados y comprobaciones iter_estatal

#iter_estatal.columns     # Columnas fueron removidas y se insertó CVE_GEO_M

In [46]:
# Análisis iter_municipal

remover_columnas_mun = ['LONGITUD', 'LATITUD', 'LON_GRAD', 'LON_MIN', 'LON_SEG', 
                        'LAT_GRAD', 'LAT_MIN', 'LAT_SEG', 'LON_DEC', 'LAT_DEC', 
                        'ALTITUD', 'TAMLOC']

#iter_municipal.isnull().all().value_counts()               # 12 columnas no tienen ningun valor
#iter_municipal.isnull().all()[iter_nacional.isnull().all() == True].index  
                                                            # Columnas que no tienen registros

#iter_municipal['ENTIDAD'].value_counts().index.min()       # Valor minimo de 01
#iter_municipal['ENTIDAD'].value_counts().index.max()       # Valor maximo de 32

#iter_municipal['MUN'].value_counts().index.min()           # Valor minimo es de 001
#iter_municipal['MUN'].value_counts().index.max()           # Valor maximo de 570 (representa el 
                                                            # numero maximo de municipios en un
                                                            # estado)


#iter_municipal[iter_municipal.LOC.isin(['0000','9999', '9998'])].NOM_LOC.value_counts()
                                                            # Las claves '9999' y '9998' no 
                                                            # representan localidades especificas

In [47]:
# Transformaciones iter_municipal

# Crear serie clave geoestadistica concatenada de ENTIDAD, MUN
clave_geo_municipal = iter_municipal['ENTIDAD'] + iter_municipal['MUN']

# Agregar serie como columna al principio del data frame
iter_municipal.insert(loc = 0, column = 'CVE_GEO_M', value= clave_geo_municipal)

# Remover columnas
iter_municipal.drop(columns=remover_columnas_mun, axis=1, inplace=True)

# Renombrar columnas
iter_municipal.rename(columns={'LOC':'CV_SUMLOC', 'NOM_LOC':'DS_SUMLOC'}, inplace = True)


C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\4206078098.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iter_municipal.drop(columns=remover_columnas_mun, axis=1, inplace=True)
C:\Users\Lorrain\AppData\Local\Temp\ipykernel_456\4206078098.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iter_municipal.rename(columns={'LOC':'CV_SUMLOC', 'NOM_LOC':'DS_SUMLOC'}, inplace = True)


In [48]:
# Resultados y comprobaciones iter_municipal

#iter_municipal.columns                          # Las columnas fueron removidas
#iter_municipal.CVE_SUMLOC.value_counts()         # Solo la clave '0000' esta presente
                                                 # en todos los municipios

#iter_municipal.loc[iter_municipal['CVE_SUMLOC'] == '0000'].CVE_GEO_M.is_unique
                                                 # Si se separa por claves CVE_SUMLOC
                                                 # solo tiene valores únicos

#iter_municipal.loc[iter_municipal['CVE_SUMLOC'] == '9998'].CVE_GEO_M.is_unique
                                                 # Valores únicos

#iter_municipal.loc[iter_municipal['CVE_SUMLOC'] == '9999'].CVE_GEO_M.is_unique
                                                 # Valores únicos


In [49]:
# Análisis iter_localidad

#iter_localidad.isnull().all().value_counts()               # No hay columnas sin valor

#iter_localidad['LOC'].value_counts().index.min()           # Valor minimo de 0001
#iter_localidad['LOC'].value_counts().index.max()           # Valor maximo de 6454


In [50]:
# Transformaciones iter_localidad

# Crear serie clave geoestadistica concatenada de ENTIDAD, MUN, y LOC
clave_geo_localidad = iter_localidad['ENTIDAD'] + iter_localidad['MUN'] + iter_localidad['LOC']

# Agregar serie como columna al principio del data frame
iter_localidad.insert(loc = 0, column = 'CVE_GEO_L', value= clave_geo_localidad)



In [51]:
# Resultados y comprobaciones iter_localidad
#iter_localidad.columns                  # Se insertó CVE_GEO_L
#iter_localidad.CVE_GEO_L.is_unique      # Todos los valores de la columna son unicos


### Cortes para SQL

 

Para almacenar los datos en bases de datos SQL, la información tiene que dividirse de manera coerente y eficiente en tablas relacionables. Existen ciertas reglas de nomenclatura, las tablas y las columnas no pueden empezar con numeros, y en general los programas usan los sufijos para relacionar tablas. Tomando en cuenta la experiencia de como se usan estos datos, la información se dividira de la siguiente manera para SQL:

* Tabla con la información censal a nivel nacional
* Tabla con informacion de claves de division politica y nombres para:
    * estados
    * municipios
    * localidades
* Tabla con información del censo 2020 para:
    * estados
    * municipios
    * localidades
* Tabla con información de coordenadas para las localidades

Las tablas de localidades se relacionaran con CVE_GEO_L, las de municipios con CVE_GEO_M y las de estados con la columna ENTIDAD. Existen las columnas necesarias dentro de la tabla de claves para relacionar los diferentes niveles exepto nacional. El poder computacional de las bases de datos SQL es bastante grande, especialmente porque su interfaz grafica es minima, por lo que no creo necesario hacer subdivisiones a la información del censo 2020 a nivel localidad.

In [52]:
col_clave_estados = iter_estatal['ENTIDAD']
col_clave_municipios = iter_municipal['CVE_GEO_M']
col_clave_localidades = iter_localidad['CVE_GEO_L']

tablas_sql = {

'ent_iter_2020' : iter_estatal.iloc[:,0:2].drop_duplicates(),
'ent_iter_censo_2020' : pd.concat([col_clave_estados, iter_estatal.iloc[:,2:]], axis=1),

'mun_iter_2020': iter_municipal.iloc[:, 0:5].drop_duplicates(),
'mun_censo_iter_2020' : pd.concat([col_clave_municipios, iter_municipal.iloc[:,5:]], axis = 1),

'loc_iter_2020' : iter_localidad.iloc[:,0:7],
'loc_id_geo_iter_2020 ' : pd.concat([col_clave_localidades, iter_localidad.iloc[:,7:18]], axis = 1),
'loc_censo_iter__2020' : pd.concat([col_clave_localidades, iter_localidad.iloc[:,18:]], axis = 1)

}


### Cortes para SIG

Los sistemas de información geografica necesitan sus tablas con ciertas caracteristicas. Los titulos de las columnas no deben de tener mas de 9 caracteres y deben ser de un solo tipo de valor (numero entero, decimal, string, o fecha) o serán consideradas como texto. Todas las columnas cumplen con estos dos requisitos.


Las tablas tampoco deben de tener multiples filas para el mismo elemento que se va a representar. Esto significa que las tablas a nivel nacional, estatal y municipal con la suma de todas las localidades, las localidades de una sola vivienda, y las localidades de dos viviendas tienen que ser separadas en varias tablas. De no ser asi, al relacionarlas con su reresentación geografica, el programa tomará el primer registro e ignorará el resto.


Los datos a nivel entidad y municipal se dividiran en las siguientes tablas para facilitar su uso en Sistemas de Información Geográfica:

    
* Tabla de estados y de municipios para:


    * La suma de todas las localidades
    * La suma de localidades con 1 vivienda
    * La suma de localidades con 2 viviendas

Para ser utilizadas en SIG, las tablas tienen que ser importadas y unidas a poligonos, lineas o puntos que representen su contenido gráficamente en formato vectorial. Los archivos mas utilizados son los que proporciona Inegi. Estos ya tienen en su tabla interna el nombre y las claves de los municipios por lo que se exportarán sin los nombres y solo con la clave necesaria para que pudenan ser unidos a los mapas que se proporcionan a travez de las columnas ENTIDAD y CVE_GEO_M.

Nacional no será separado porque al solo ser un poligono no se usará para el procesamiento y analisis espacial. Esto sería diferente si la base de datos tuviera información de dos o mas paises.

In [53]:
# Dividir estados por tipo de suma de localidades

tablas_estados = {
    '2020_iter_tot_ent' : iter_estatal.loc[iter_estatal['CV_SUMLOC'] == '0000'],
    '2020_iter_l1v_ent' : iter_estatal.loc[iter_estatal['CV_SUMLOC'] == '9998'],
    '2020_iter_l2v_ent' : iter_estatal.loc[iter_estatal['CV_SUMLOC'] == '9999'],
}



In [54]:
#iter_estatal['CV_SUMLOC'].value_counts()      # Todas las claves tienen la misma cantidad de registros

In [55]:
# Dividir municipios por tipo de suma de localidades
tablas_municipios = {
    '2020_iter_tot_mun' : iter_municipal.loc[iter_municipal['CV_SUMLOC'] == '0000'],
    '2020_iter_l1v_mun' : iter_municipal.loc[iter_municipal['CV_SUMLOC'] == '9998'],
    '2020_iter_l2v_mun' : iter_municipal.loc[iter_municipal['CV_SUMLOC'] == '9999']
}


In [56]:
#iter_municipal['CV_SUMLOC'].value_counts()[0]
#iter_municipal['CV_SUMLOC'].value_counts()[0] == len(municipios)

Las localidades no solo se representan como puntos, los marcos geoestadisticos que proporciona INEGI tienen archivos que representan muchas de las localidades en formato de poligono. Las que todavia no tienen poligono son representadas en otro archivo por puntos. Las tablas que seran exportadas son:
* Tabla solamente con la informacion de claves, nombres y geolocalización
* Tabla con toda la información censal
* Tablas con la información censal dividia por temas de INEGI
    * Población
    * Estructura por Edad y Sexo
    * Fecundidad
    * Migración
    * Etnicidad
    * Discapacidad
    * Educación
    * Características económicas
    * Servicios de salud
    * Situación conyugal
    * Religión
    * Hogares censales
    * Vivienda

Son muchas las maneras en que se puede dividir la información, y muchas veces hay que tomar uno o varios campos de uno o varios temas. Para eso esta la tabla con toda la información censal. Muchas veces solo se necesita un tema especifico, para estas veces, y por lo pesado de los archvios cuando se le agrega una representacion grafica, fueron separados por tema. Esta es solo una propuesa de división basada en mi experiencia. Hay otras formas según diferentes necesidades de análisis.

In [57]:

columnas_tablas_loc = {
    '2020_iter_localidades' : list(raw_censo_2020.columns[0:6]) + ['LON_DEC', 'LAT_DEC'],
                                                                            # Desde ENTIDAD a NOM_LOC
                                                                            # + LON_DEC y LAT_DEC
    '2020_iter_id_geo_loc' : list(raw_censo_2020.columns[6:17]),            # Desde LONGITUD hasta ALTITUD
    '2020_iter_poblacion_loc' : list(raw_censo_2020.columns[17:64]),        # Desde POBTOT hasta POB65_MAS
    '2020_iter_estructura_pob_loc' : list(raw_censo_2020.columns[64:118]),  # Desde P_0A4 hasta P_85YMAS_M
    '2020_iter_fecundiad_loc' :  list(raw_censo_2020.columns[118:119]),     # Solo PROM_HNV
    '2020_iter_migracion_loc' : list(raw_censo_2020.columns[119:131]),      # Desde PNACENT hasta PRESOE15_M
    '2020_iter_etnicidad_loc' : list(raw_censo_2020.columns[131:147]),      # Desde P3YM_HLI hasta POB_AFRO_M
    '2020_iter_discapacidad_loc' : list(raw_censo_2020.columns[147:163]),   # Desde PCON_DISC hasta PSIND_LIM
    '2020_iter_educacion_loc' : list(raw_censo_2020.columns[163:205]),      # Desde P3A5_NOA hasta GRAPROES_M
    '2020_iter_economia_loc' : list(raw_censo_2020.columns[205:217]),       # Desde PEA hasta PDESOCUP_M
    '2020_iter_salud_loc' : list(raw_censo_2020.columns[217:227]),          # Desde PSINDER hasta PAFIL_OTRAI
    '2020_iter_conyugal_loc' : list(raw_censo_2020.columns[227:230]),       # Desde P12YM_SOLT hasta P12YM_SEPA
    '2020_iter_religion_loc' : list(raw_censo_2020.columns[230:234]),       # Desde PCATOLICA hasta PSIN_RELIG
    '2020_iter_hogares_loc' : list(raw_censo_2020.columns[234:240]),        # Desde TOTHOG hasta PHOGJEF_M
    '2020_iter_vivienda_loc' : list(raw_censo_2020.columns[240:] )          # Desde VIVTOT hasta TAMLOC
}

In [58]:
# Diccionario de tablas a nivel localidad
tablas_localidad = {}

# Censo completo por localidad
tablas_localidad['2020_iter_censo_loc'] = iter_localidad

# Iter a nivel localidad por temas de INEGI segun las columnas previamente definidas
for nombre_tabla, lista_columnas in columnas_tablas_loc.items():
    
    columnas_tabla = ['CVE_GEO_L'] + lista_columnas
    tablas_localidad[nombre_tabla] =  iter_localidad.loc[:, columnas_tabla]

In [59]:
#tablas_localidad['2020_iter_salud_loc']
#tablas_localidad['2020_iter_id_geo_loc']

## Agregar indicadores creados al descriptor

Los indicadores creados serán agragados al descriptor de datos original y despues serán exportados con los diferentes cortes.

In [60]:
# Diccionario de indicadores generados
indicadores_agregados = {

    'LON_GRAD' : {
        'indicador' : 'Grados de longitud',
        'descripcion' : 'Grados de la longitud geográfica.',
        'rangos' : '-180...180',
        'longitud' : 3
    },

    'LON_MIN' : {
        'indicador' : 'Minutos de longitud',
        'descripcion' : 'Minutos de la longitud geográfica.',
        'rangos' : '-59...59',
        'longitud' : 2
    },

    'LON_SEG' : {
        'indicador' : 'Segundos de longitud',
        'descripcion' : 'Segundos de la longitud geográfica.',
        'rangos' : '-59.999...59.999',
        'longitud' : 9
    },

    'LAT_GRAD' : {
        'indicador' : 'Grados de latitud',
        'descripcion' : 'Grados de la latitud geográfica.',
        'rangos' : '-90...90',
        'longitud' : 2
    },

    'LAT_MIN' : {
        'indicador' : 'Minutos de latitud',
        'descripcion' : 'Minutos de la latitud geográfica.',
        'rangos' : '-59...59',
        'longitud' : 2
    },

    'LAT_SEG' : {
        'indicador' : 'Segundos de latitud',
        'descripcion' : 'Segundos de la latitud geográfica.',
        'rangos' :  '-59.999...59.999',
        'longitud' :  9
    },

    'LON_DEC' : {
        'indicador' : 'Longitud grados decimales',
        'descripcion' : 'Longitud en grados decimales.',
        'rangos' : '-180...180',
        'longitud' : 9
    },

    'LAT_DEC' : {
        'indicador' : 'Latitud grados decimales',
        'descripcion' : 'Latitud en grados decimales.',
        'rangos' : '-90...90',
        'longitud' : 9
    },

    'CV_SUMLOC' : {
        'indicador' : 'Clave de Suma de localidades según número de vivienda y total',
        'descripcion' : 'Clave que representa la suma del total de viviendas (0000) y el total de las localidades de una (9998) y dos viviendas (9999).',
        'rangos' : '0000,9998...9999',
        'longitud' : 4
    },

    'DS_SUMLOC' : {
        'indicador' : 'Descripcion de la suma de localidades según número de vivienda y total',
        'descripcion' : 'DescripcionSuma de las localidades de una (9998) y dos viviendas (9999) y del total de viviendas (0000).',
        'rangos' : 'Alfanumérico',
        'longitud' : 50
    },
    
    'GVE_GEO_L' : {
        'indicador' : 'Clave geoestadística de la localidad',
        'descripcion' : 'Clave geoestadística de la localidad concatenada con las claves de estado, municipio y loalidad.',
        'rangos' : 'Alfanumérico',
        'longitud'  : 9
    },

    'CVE_GEO_M' : {
        'indicador' : 'Clave geoestadística del municipio',
        'descripcion' : 'Clave geoestadística del municipio concatenada con las claves de estado y municipio.',
        'rangos' : 'Alfanumérico',
        'longitud' : 5
    }

}



In [61]:
# Agregar indicadores a descriptor
ultimo_indice = 0

for indicador, dic_columna in indicadores_agregados.items():
    ultimo_indice = ultimo_indice + 1
    a_unir = [ultimo_indice, 
              dic_columna['indicador'], 
              dic_columna['descripcion'],
              indicador,
              dic_columna['rangos'],
              dic_columna['longitud']]
    descriptor.loc[len(descriptor.index)] = a_unir

In [66]:
# REsultado

#descriptor.iloc[-12:]

## Exportar las tablas

In [198]:
entidad = '30'
#municipio = '193'
contiene = '[Tt]embla'

iter_localidad[(iter_localidad.ENTIDAD == entidad) & \
               #(iter_localidad.MUN == municipio) & \
               (iter_localidad.NOM_LOC.str.contains(contiene, regex=True))]

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,LON_GRAD,LON_MIN,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
169316,30,Veracruz de Ignacio de la Llave,015,Angel R. Cabada,0187,Las Tembladeras,"95°23'30.061"" W","18°39'00.146"" N",-95,-23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
175628,30,Veracruz de Ignacio de la Llave,092,Xico,0025,Tembladeras,"97°06'50.632"" W","19°30'43.892"" N",-97,-6,...,70,1,1,0,0,8,49,118,7,3
186815,30,Veracruz de Ignacio de la Llave,193,Veracruz,0143,Rancho Tembladeras,"96°09'28.587"" W","19°08'14.088"" N",-96,-9,...,4,0,1,0,0,0,0,4,0,1


In [181]:
iter_localidad[iter_localidad.NOM_LOC.str.contains('aderas')]

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,LON_GRAD,LON_MIN,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
787,01,Aguascalientes,003,Calvillo,0073,El Refugio (Las Praderas),"102°45'24.197"" W","21°48'45.857"" N",-102,-45,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
1100,01,Aguascalientes,005,Jesús María,0326,Las Laderas,"102°32'00.241"" W","21°49'36.565"" N",-102,-32,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
7504,02,Baja California,006,San Quintín,0660,Praderas de San Quintín,"115°57'05.239"" W","30°32'20.665"" N",-115,-57,...,48,13,28,3,1,6,1,35,1,1
8879,03,Baja California Sur,003,La Paz,1348,Las Praderas,"110°12'17.816"" W","23°29'21.917"" N",-110,-12,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
9860,03,Baja California Sur,008,Los Cabos,0759,Las Praderas,"109°44'06.293"" W","23°07'24.930"" N",-109,-44,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184499,30,Veracruz de Ignacio de la Llave,172,Texistepec,0091,Las Praderas,"94°46'34.328"" W","17°52'23.501"" N",-94,-46,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
186815,30,Veracruz de Ignacio de la Llave,193,Veracruz,0143,Rancho Tembladeras,"96°09'28.587"" W","19°08'14.088"" N",-96,-9,...,4,0,1,0,0,0,0,4,0,1
188548,31,Yucatán,013,Conkal,0149,Praderas del Mayab,"89°31'56.370"" W","21°05'52.560"" N",-89,-31,...,19,17,9,9,7,0,0,2,0,1
191251,32,Zacatecas,006,Cañitas de Felipe Pescador,0029,Las Praderas (J. Jesús Hernández Rodríguez) [R...,"102°46'16.498"" W","23°33'40.610"" N",-102,-46,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1


In [175]:
entidad = '30'
municipio = '182'
localidad = '0020'

iter_localidad[(iter_localidad.ENTIDAD == entidad) & \
               (iter_localidad.MUN == municipio) & \
               (iter_localidad.LOC == localidad )]

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,LON_GRAD,LON_MIN,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC


In [103]:
iter_localidad.NOM_LOC.str.startswith(starts_with).value_counts()

False    189153
True        279
Name: NOM_LOC, dtype: Int64

In [161]:
iter_localidad[iter_localidad.NOM_MUN.str.contains('Cosautl')]

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,LON_GRAD,LON_MIN,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
171199,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0001,Cosautlán de Carvajal,"96°59'28.912"" W","19°19'57.517"" N",-96,-59,...,1089,324,173,41,49,75,231,938,43,5
171200,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0002,La Gloria,"97°00'16.544"" W","19°20'56.915"" N",-97,0,...,40,1,0,0,0,9,42,77,9,2
171201,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0003,Huehuetecpan,"97°00'03.051"" W","19°19'11.481"" N",-97,0,...,85,3,5,1,1,8,42,121,6,3
171202,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0004,Juanantontla,"97°01'04.283"" W","19°20'12.010"" N",-97,-1,...,24,2,0,0,0,13,32,52,12,1
171203,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0005,Limones,"96°55'26.666"" W","19°20'12.819"" N",-96,-55,...,463,133,30,6,13,31,117,483,16,4
171204,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0006,Las Lomas,"96°59'42.217"" W","19°21'14.315"" N",-96,-59,...,54,6,3,0,0,3,22,67,3,2
171205,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0008,Palzoquitipan,"97°01'32.648"" W","19°20'43.476"" N",-97,-1,...,35,4,0,0,0,8,24,55,8,2
171206,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0010,Piedra Parada,"96°57'33.853"" W","19°20'49.142"" N",-96,-57,...,307,30,16,2,2,40,117,411,31,4
171207,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0011,La Reforma,"96°56'36.067"" W","19°19'28.797"" N",-96,-56,...,117,46,4,2,2,9,24,94,9,3
171208,30,Veracruz de Ignacio de la Llave,046,Cosautlán de Carvajal,0012,San José Tlapéxcatl,"96°59'06.839"" W","19°19'00.957"" N",-96,-59,...,61,7,2,0,0,7,33,87,5,2


In [100]:
iter_localidad.NOM_LOC

7                           Aguascalientes
8                           Granja Adelita
9                                Agua Azul
10                  Los Arbolitos [Rancho]
11        Ardillas de Abajo (Las Ardillas)
                        ...               
195656     Los Trigos (Mesa de los Trigos)
195657                         Mesa Grande
195658                          San Isidro
195659               San Miguel Tepetitlán
195660                          San Rafael
Name: NOM_LOC, Length: 189432, dtype: string

para las tablas de municipios y localidades respectivamente. En el caso de las localidadees la columna contendrá solo valores unicos ya que solo hay un registro por localidad. Para que solo haya un registro por clave geo en el nivel municipal habrá que separar la sumatoria total y las sumatorias de localidades de 1 y 2 viviendas.

## Pruebas...

In [9]:

column_dtype = {}
for column in raw_censo_2020.columns:

    if column in columnas_string:
        column_dtype[column] = 'string'

    elif column in columnas_decimal:
        column_dtype[column] = 'Float64'

    else:
        column_dtype[column] = 'Int64'
